## Import modules

In [ ]:
import pandas as pd
import numpy as np
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import urllib.request
import re

## Datasets

- Read in dataset containing MITRE ATT&CK groups, groupids and techniques
    - File was pulled from MITRE
    - Data/Cells associated with technique columns were all set to 0
       - This was done to create a template.
       - Will fill with 1 after scaping MITRE ATT&CK group sites for groups matching specific techniques
- Create `df_groups` dataframe for CSV file

In [ ]:
df_groups = pd.read_csv("MitreAttack_Dataset.csv")
print(df_groups)

- In the next section, we open a file called *Groups.txt*
    - This file contains a text list of all MITRE ATT&CK attack groups
    - We then iterate through the file, adding groups to the Groups list
    - We print the Groups list to make sure that it is correct

In [ ]:
Groups = []
with open("Groups.txt", "r") as GroupsFile:
   for line in GroupsFile:
      Groups.append(line)
Groups = [group.replace('\n','') for group in Groups]
print(Groups)

In [ ]:
#df_groups = pd.DataFrame(Groups, columns=['Groups'])
#print(df_groups)

- The URL: *https://attack.mitre.org/groups/<groupID>* provides a web page containing a Groups attack techniques
- In the next section, we create a user agent that will be provided in the header to connect to the above URL
- We then iterate through the Groups list and build the http header to connect to the above URL
    - We look for the regular expressions associated with the techniques and create a list called techniques with all the scraped techniques
- Finally, we iterate through our techniques list
    - Here, we look for the associated group and technique
       - if there is a match, we set the technique for the associated group to 1 in the pandas dataframe

    

In [ ]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
for group in Groups:
   url = "https://attack.mitre.org/groups/" + group
   opener = urllib.request.build_opener()
   opener.addheaders = [('User-agent', USER_AGENT)]
   response = urllib.request.urlopen(url)
   html_content = response.read()
   pattern = re.compile("T[0-9]{4}\/[0-9]{3}\">\.[0-9]{3}") 
   techniques = re.findall(pattern,str(html_content))
   techniques = [technique.replace('/','.') for technique in techniques]
   techniques = [re.sub(r'\"\>\.[0-9]{3}','', technique) for technique in techniques]
   techniques = list(set(techniques))
   for technique in techniques:
      df_groups.loc[df_groups["GroupID"] == group, technique] = 1
#      if technique not in techniques:
#        df_groups.loc[df_groups["GroupID"] == group, technique] = 0

## Things to do in next cell

- Because each MITRE ATT&CK group is not an enterprise attack group (some may be mobile)
    - We will pick up new columns with NaNs in the rows
    - We need to delete the columns with all NaNs
       - Drop columns where all values are NaN (solved)
- Next we need to drop all rows with zeros in the techniques
    - All zeros will affect our network science affiliation builds
      - Delete any rows where all techniques are 0's  
- Finally, we write out the dataframe created into a .csv file
    - This will be imported into networkx later
       - Note, we could just use the dataframe itself for networkx
       - Write out the dataframe to a CSV file

In [ ]:
# Drop columns where all values are NaN
df_groups = df_groups.dropna(axis=1, how='all')

# Drop rows where all techniques in a row are 0
#print(techniques)
df_groups = df_groups.loc[(df_groups[techniques] != 0).any(axis=1)]

#print(df_test)


In [ ]:

#df_groups
df_groups.to_csv("MitreGroups_Dataset3.csv", index = False)


## Network Science

### If "MitreGroups_Dataset3.csv" dataset already exists, start here

## Import networkx and setup visualization parameters

In [ ]:
import random
import networkx as nx

# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (7.5, 7.5),
    'axes.spines.right': True,
    'axes.spines.left': True,
    'axes.spines.top': True,
    'axes.spines.bottom': True})
# Seed random number generator
from numpy import random as nprand
seed = hash("Network Science in Python") % 2**32
nprand.seed(seed)
random.seed(seed)

- We need to create an affiliation network
    - Open up the .csv dataset recently created
    - Parse each row (except the header)
        - Create a list called `parts` containing values for each row
        - Add groups from each row to a list called `groupids`
        - add each groupids to the set called `groups`
        - Enumerate over all columns in each row (except the groupid column) looking for the value of 1
           - Here, 1 is associated with the technique that the group actually uses
           - If the value is not 0, we add the groupid and technique as edges, and value as a weight attribute to the B network edges
- Import the bipartite method from networkx (To create an affiliation network)
- Create the G affiliation network from B network and groups set

In [ ]:
# Create empty affiliation network and list of MITRE ATT&CK groups
B = nx.Graph()
groups = set()
# Load data file into network
from pathlib import Path
data_dir = Path('.')
with open(data_dir / 'MitreGroups_Dataset3.csv') as f:
    # Parse header
    events = next(f).strip().split(",")[1:]
    #print(events)
    # Parse rows
    for row in f:
        #print(row)
        parts = row.strip().split(",")
        #print(parts)
        groupids = parts[0]
        #print(groupids)
        groups.add(groupids)
        #print(groups)
        for j, value in enumerate(parts[1:]):
            #print(j, value)
            if value != "0":
                B.add_edge(groupids, events[j], weight=int(value))
                B.nodes[groupids]["bipartite"] = 0
                B.nodes[events[j]]["bipartite"] = 1
                #print(B.edges(data=True))
# Project into group-to-group co-affilation network

from networkx import bipartite
G = bipartite.projected_graph(B, groups)
G.nodes(data=True)
#B = B.subgraph(list(nx.connected_components(B))[0])

In [ ]:
print(B.edges(data=True))

In [ ]:
# Get node sets
#print(B.nodes(data=True))
attackers = [v for v in B.nodes if B.nodes[v]["bipartite"] == 0]
#print(pollinators)
ma_techniques = [v for v in B.nodes if B.nodes[v]["bipartite"] == 1]
print(attackers)

- Now that we have our affiliation network, we can look at centralities
- First, we look at betweeness centrality
    - Betweenness Centrality is based on the assumption that...
        - The greater the number of shortest paths pass through a node (or edge)...
           - The more it acts as a broker (or bridge)
    - To calculate betweenness centrality:
        - The shortest paths between each pair of nodes are found
        - The betweenness centrality value for a node or edge is just the number of these paths that pass through it
- High betweenness centralities suggest...
     - These attack groups are skilled or mature among attackers
        - Suggesting  Nation-State or a high-level criminal organization
- However, network analysis alone cannot reveal why these groups are mature in their methods.
    - This requires richer data and methods
- However, looking at other network measures can still help shed some light on the role these attack groups play

In [ ]:
betweenness = nx.betweenness_centrality(G, normalized=False)
sorted(betweenness.items(), key=lambda x:x[1], reverse=True)[0:10]

- The more well-connected a node:
    - The higher the eigenvector centrality
- In NetworkX, the eigenvector_centrality() function can be used to calculate eigenvector centrality
- As with other centrality measures, this function returns a dict that maps node IDs to centrality values
- The following example applies the function to the attackers network
- Prints the top 10 hubs in the network:

- Just as before, these are all notable attackers from the MITRE ATT&CK framework groups
- However, some attackers with high eigenvector centrality don't have particularly high betweenness centrality
    - Actually, these attack groups don't have high eigenvector centraility
       - .10 ?
- Groups with high eigenvector centrality create many short paths between others
    - But, not necessarily the *shortest* paths

In [ ]:
eigenvector = nx.eigenvector_centrality(G)
sorted(eigenvector.items(), key=lambda x:x[1], reverse=True)[0:10]

- Nodes with high closeness have, on average, short paths to other nodes
    - Can be helpful for understanding depth in capabilities
    
- The following example uses the NetworkX `closeness_centrality()` function
    - Calculates the closeness centrality values for the attack groups network and displays the top 10

- Nearly every group in this list appears in at least one of the other top 10 lists
    - Only G0069 and G0010 don't appear in all three
    - G0032 comes out on top of all three
       - Clearly more active or mature than the others
       - Not less important
- Important lesson is:
    - The important structural roles played by brokers and hubs in a network are easily and often obscured

In [ ]:
closeness = nx.closeness_centrality(G)
sorted(closeness.items(), key=lambda x:x[1], reverse=True)[0:10]

- We will now look at the relationships between a node's neighbors
    - rather than just the node itself
- Often useful to consider whether a node's neighbors tend to be connected to each other
    - In an attack network, this question translates to asking whether the attacker of an attacker is also similar to  corresponding attacker in regards to techniques used.
       - This is a property known as **transitivity** (a math thing)
- The result of such relationships are triangles:
    - Three nodes, all mutually connected
       - Fully meshed
- The tendency for such triangles to arise is called clustering
    - When strong clustering is present, it often suggests robustness, and redundancy in a network
       - If one edge disappears, a path still exists via the other two
- Clustering is measured via the **local clustering coefficient**
    - Defines as the fraction of all pairs of a node's neighbors that have an edge between them
    
    
- In NetworkX, the number of triangles between a node and its neighbors can be calculated using the `triangles` function
    - The below output tells us that G0032, G0128, G0007, G0094, G0046, G0016 and G0059 have strong similarites in techniques
       - Are these groups associated somehow? Perhaps by country? Further analysis is needed

In [ ]:
triangles = nx.triangles(G)
sorted(triangles.items(), key=lambda x:x[1], reverse=True)[0:10]

- Remember...
    - Clustering is measured via the local clustering coefficient
    - Defines as the fraction of all pairs of a node's neighbors that have an edge between them
    
- Now, we can use the `clustering()` function to find the local clustering coefficient for these nodes

- G0032, G0128, G0007, G0094, G0046, G0016 and G0059 shows up as having the highest centrality in terms of triangles
- These top nodes have local clustering coefficients in roughly 85-86% range
- While, nodes in the network span the entire 0% - 100% range
    - The seven attackers common to the top 10 lists for all centrality measures have local clustering coefficients of 85-86%
- If an attacker's local clusting coefficient is low:
    - Suggests their techniques are not well established
- If the coefficient is high
    - Suggests that an attacker's techniques are well established or mature
- So, the most central attackers have absolute numbers of triangles and high local clustering coefficients

In [ ]:
clustering = nx.clustering(G)
[(x, clustering[x]) for x in sorted(groups, key=lambda x:eigenvector[x], reverse=True)[0:10]]

## Visualization

- We will now create visualizations of our attacker affiliation network
- First, we will plot/draw out the current network using a spring layout
- We define some basic parameter for the plot
- We also save the plot to a file

In [ ]:
# Create co-affiliation network
G = bipartite.projected_graph(B, ma_techniques)
#print(G.edges(data=True))
# Create figure
plt.figure(figsize=(24,24))
# Calculate layout
pos = nx.spring_layout(G, k=0.5)
# Draw edges, nodes, and labels
nx.draw_networkx_edges(G, pos, width=3, alpha=0.2)
nx.draw_networkx_nodes(G, pos, node_color="#bfbf7f", node_shape="h", node_size=10000)
nx.draw_networkx_labels(G, pos)
plt.savefig('techniques_affiliation.png', dpi=150)

- In the preceding unweighted projections, considerable information is lost concerning structure of network
- An edge might mean that two nodes have one common affiliation or hundreds
- Weighted projections help us capture some of that lost information
- Weighted projections turn a weighted or un-weighted affiliation network into a weighted co-affiliation network
    - Some of the structural information lost is recaptured in the edge weights
- A common way to calculate edge weights is simply by counting the number of neighbors
- This technique can be interpreted as...
    - Counting the number of paths between two nodes in the original affiliation network
- In NetworkX, projection is achieved using the weighted_projected_graph() function
    - Seen in the following code
       - Computed weight is stored in the weight edge attribute
           - Here, the computed weight is the number of techniques the attack groups have in common
              - This could help with attribution of unknown groups
       - Note: we just print out the first edge below

In [ ]:
G = bipartite.weighted_projected_graph(B, attackers)
list(G.edges(data=True))[40]

In [ ]:
# Create co-affiliation network
#G = bipartite.projected_graph(B, groups)
# Create figure
plt.figure(figsize=(24,24))
# Calculate layout
#pos = nx.shell_layout(G)
#pos = nx.circular_layout(G)
pos = nx.spring_layout(G, k=0.5)
# Draw edges, nodes, and labels
nx.draw_networkx_edges(G, pos, width=3, alpha=0.2)
nx.draw_networkx_nodes(G, pos, node_color="#bfbf7f", node_shape="h", node_size=10000)
nx.draw_networkx_labels(G, pos)
plt.savefig('attackers_affiliation.png', dpi=150)

- Edge weights can also be calculated using a similarity measure such a the *Jaccard Index*
    - The Jaccard Index for two nodes is the...
       - Number of common neighbors divided by the number of nodes that neighbor either of the nodes, 
          - and ranges from 0 (no common neighbors) to 1 (all neighbors are common)
       - The `overlap_weighted_projection_graph()` function creates a projection using the Jaccard index
    - Following code calculates such a projection for the attacker (group) network
       - Visualizes edge weights using a color gradient

In [ ]:
# Create co-affiliation network
G = bipartite.overlap_weighted_projected_graph(B, groups)
# Get weights
edge_weight = [G.edges[e]['weight'] for e in G.edges]
# Create figure
plt.figure(figsize=(30,30))
# Calculate layout
pos = nx.spring_layout(G, weight='edge_weight', k=0.5)
# Draw edges, nodes, and labels
nx.draw_networkx_edges(G, pos, edge_color=edge_weight, edge_cmap=plt.cm.Blues, width=6, alpha=0.5)
nx.draw_networkx_nodes(G, pos, node_color="#9f9fff", node_size=6000)
nx.draw_networkx_labels(G, pos)
plt.savefig('attacker_co-affiliation_network.png', dpi=150)

- The preceding code uses the `edge_color` parameter of `draw_network_edges()` to color edges based on their projected weight
    - allows the strength of the connections to be visualized
- The diagram uncovers some additional information about the attacker-attacker network
    - While most nodes have many neighbors
       - the weight of those edges is relatively low
    - Some nodes have fewer but stronger connections
       - Shows isolated groups of attackers that have much in common with each other
- Ultimately, the type of projection to use depends on the nature of the network data and the question yo hope to answer

- Next we are going to focus on attackers that contain matching techniques
    - So using the `bipartite.weighted_projected_graph()` method we calculate the number of techniques that attackers (nodes) have in common
       - Happens automagically using this method
- We then create a new network named `network` and add all edges from the `G` network that have 30 or more techniques in common
    - This is stored in the weight attribute

In [ ]:
G = bipartite.weighted_projected_graph(B, attackers)
#network.clear()
network = nx.Graph()
#print(G.edges(data=True))
for a, g, w in G.edges(data=True):
   if w['weight'] >= 30:
      #print(a, g)
      network.add_edge(a, g)
      network.edges[a,g]["weight"] = w['weight']
      #network.set_edge_attributes(network, G.edges)

- Let's take a look at those edges in the `network` network

In [ ]:
print(network.edges(data=True))

- Now that we have a network that is narrowed down to edges that have higher weights (attackers that have the highest number of techniques in common)
    - We can be begin visualizing the network and attacker affiliations
-First, we create some variables associated with weights for drawing lines
    - edge_weight: Used to define line colors
    - edge_width: Used to define line width
    - edge_color: Used to define line color in our multi-plot visualization
- We then draw our edges and nodes
    - We save the output diagram to a file

In [ ]:
edge_weight = [network.edges[e]['weight'] for e in network.edges]
edge_width = [network.edges[e]['weight'] / 5 for e in network.edges]
edge_color = [network.edges[e]['weight'] * 10 for e in network.edges]
# Create figure
plt.figure(figsize=(5,5))
# Calculate layout
#pos = nx.shell_layout(G)
pos = nx.circular_layout(network)
#pos = nx.spring_layout(network, k=0.5)
# Draw edges, nodes, and labels
nx.draw_networkx_edges(network, pos, edge_color = edge_weight, width = edge_width, alpha=0.2)
nx.draw_networkx_nodes(network, pos, node_color="#bfbf7f", node_shape="h", node_size=1000)
nx.draw_networkx_labels(network, pos)
plt.savefig('attacker_weights.png', dpi=150)

- Finally, we draw 4 separate layouts to find a good visualization
    - Spring Layout
    - Random Layout
    - Spiral Layout
    - Circular Layout

In [ ]:
# Draw our graph
#Define our Graph Plot Layouts
fig_size = (10,7)
layouts = (nx.spring_layout, nx.random_layout, nx.spiral_layout, nx.circular_layout)
title = ("Force-directed", "Random", "Spiral", "Circular")
## Create our Plots
# Create 4 subplots with the figure size based on graph size
_, plot = plt.subplots(2, 2, figsize=fig_size)
subplots = plot.reshape(1, 4)[0]
# Draw a plot for each layout
for plot, layout, title in zip(subplots, layouts, title):
    pos = layout(network)
    # Use seed to ensure re-creation of same visuals 
    seed = hash("Network Science in Python") % 2**32
    nprand.seed(seed)
    random.seed(seed) 
    # Draw nodes and edges
    nx.draw_networkx_edges(network, pos, ax=plot, width=1.0, style="solid", edge_color=edge_color, edge_cmap=plt.cm.Greys, alpha=1.0)
    nx.draw_networkx_nodes(network, pos, ax=plot, node_size = 600, node_color="yellow", node_shape="o")
    # Draw labels
    nx.draw_networkx_labels(network, pos, ax=plot, font_color="red", font_size=8)
    plot.set_title(title)
    plt.savefig('attacker_visualizations.png', dpi=150)

# Draw with tight layout https://matplotlib.org/tutorials/intermediate/tight_layout_guide.html
plt.gca().margins(0.20, 0.20)
plt.tight_layout()
plt.show()